In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
#nltk.download('wordnet')

In [2]:
class Data:
    def __init__(self, limit=5000):
        self.data_brut = pd.read_json("train.jsonl", lines=True)[["texte_annonce", "cal_réponse_signalement"]]
        self.data = self.data_brut.iloc[:limit]

    def get_data(self):
        return self.data

    def rien(self, x):
        pass

    def racinisation(self, text_column):
        stemmer = SnowballStemmer("french")
        self.data.loc[:, text_column] = self.data[text_column].apply(lambda x: ' '.join([stemmer.stem(word) for word in str(x).split()]))

    def lemmatisation(self, text_column):
        lemmatizer = WordNetLemmatizer()
        self.data.loc[:, text_column] = self.data[text_column].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in str(x).split()]))

    def vectorisation_simple(self, text_column):
        vectorizer = CountVectorizer()
        vect_data = vectorizer.fit_transform(self.data[text_column])
        vect_df = pd.DataFrame(vect_data.toarray(), columns=vectorizer.get_feature_names_out())
        self.data = pd.concat([vect_df, self.data.iloc[:, -1]], axis=1)

    def vectorisation_ponderee(self, text_column):
        vectorizer = TfidfVectorizer()
        vect_data = vectorizer.fit_transform(self.data[text_column])
        vect_df = pd.DataFrame(vect_data.toarray(), columns=vectorizer.get_feature_names_out())
        self.data = pd.concat([vect_df, self.data.iloc[:, -1]], axis=1)

    def reduction_svd(self, n_components=100):
        svd = TruncatedSVD(n_components=n_components)
        svd_result = svd.fit_transform(self.data.iloc[:, :-1])
        self.data = pd.concat([pd.DataFrame(svd_result), self.data.iloc[:, -1]], axis=1)

    def reduction_nmf(self, n_components=100):
        nmf = NMF(n_components=n_components)
        nmf_result = nmf.fit_transform(self.data.iloc[:, :-1])
        self.data = pd.concat([pd.DataFrame(nmf_result), self.data.iloc[:, -1]], axis=1)


In [14]:
class Classifieur:
    def __init__(self, data):
        # Chargement et découpage des données
        self.chargement(data)
        self.decoupage()

    def chargement(self, data):
        # Chargement des données et séparation X, y
        self.data = data
        self.X = data.iloc[:, :-1]  # Toutes les colonnes sauf la dernière (X)
        self.y = data.iloc[:, -1]   # Dernière colonne (y)

    def decoupage(self):
        # Découpage en jeu d'entrainement et test
        self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(self.X, self.y, test_size=0.6, random_state=0)

    def entrainement(self):
        pass

    def rien(self):
        pass
    
    def taux_reussite(self):
        # Calcul du taux de réussite
        y_pred = self.classifier.predict(self.X_test)
        accuracy = accuracy_score(self.Y_test, y_pred)
        return f"{accuracy:.3f}"

    def liste_reussite(self):
        pass

    def graph_train_vs_test(self, titre_graph, axe_x, nom_x, list_reussite_train, list_reussite_test):
        # Affichage d'un graphique des taux de réussite pour l'entrainement et la validation
        plt.figure(figsize=(10, 6))
        plt.plot(axe_x, list_reussite_train, label=f"% réussite entrainement", marker='o')
        plt.plot(axe_x, list_reussite_test, label=f"% réussite validation", marker='o')
        plt.xlabel(nom_x)
        plt.ylabel("Taux de réussite")
        plt.title(titre_graph)
        plt.legend()
        plt.show()


In [15]:
from sklearn.naive_bayes import GaussianNB

class BayesienNaif(Classifieur):
    def __init__(self, data):
        super().__init__(data)

    def entrainement(self):
        self.classifier = GaussianNB()
        self.classifier.fit(self.X_train, self.Y_train)

In [16]:
from sklearn.linear_model import LogisticRegression

class RegressionLogistique(Classifieur):
    def __init__(self, data):
        super().__init__(data)

    def entrainement(self):
        self.classifier = LogisticRegression(max_iter=1000, random_state=0)
        self.classifier.fit(self.X_train, self.Y_train)


In [17]:
from sklearn.neighbors import KNeighborsClassifier

class KPlusProchesVoisins(Classifieur):
    def __init__(self, data):
        super().__init__(data)

    def entrainement(self, n_neighbors=3):
        self.classifier = KNeighborsClassifier(n_neighbors=n_neighbors)
        self.classifier.fit(self.X_train, self.Y_train)


In [18]:
from sklearn.tree import DecisionTreeClassifier

class ArbreDeDecision(Classifieur):
    def __init__(self, data):
        super().__init__(data)

    def entrainement(self):
        self.classifier = DecisionTreeClassifier(random_state=0)
        self.classifier.fit(self.X_train, self.Y_train)


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

class ForetAleatoire(Classifieur):
    def __init__(self, data):
        super().__init__(data)

    def entrainement(self, n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features='sqrt'):
        self.classifier = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            random_state=0
        )
        self.classifier.fit(self.X_train, self.Y_train)
    
    def recherche_hyperparametres(self, param_grid):
        # Recherche des meilleurs hyperparamètres avec GridSearchCV
        grid_search = GridSearchCV(
            RandomForestClassifier(random_state=0),
            param_grid,
            cv=5,  # Validation croisée 5-fold
            scoring='accuracy',
            n_jobs=-6  # Utilisation maximale des cœurs CPU
        )
        grid_search.fit(self.X_train, self.Y_train)
        self.classifier = grid_search.best_estimator_
        print("Meilleurs hyperparamètres :", grid_search.best_params_)

In [20]:
def possibilite():
    results = []  # Liste pour stocker les résultats
    
    for x in ["lemmatisation", "racinisation","rien"]:
        for y in ["vectorisation_simple", "vectorisation_ponderee"]:
            data = Data()
            getattr(data, x)("texte_annonce")
            getattr(data, y)("texte_annonce")

            if y == "vectorisation_ponderee":
                data.reduction_nmf()
                z = "nmf"
            else:
                data.reduction_svd()
                z = "svd"

            # Parcours des classifieurs
            for i in [BayesienNaif, RegressionLogistique, KPlusProchesVoisins, ArbreDeDecision, ForetAleatoire]:
                classifieur = i(data.get_data())
                classifieur.entrainement()
                taux_reussite = classifieur.taux_reussite()
                
                # Ajout des résultats à la liste sous forme de tuple
                results.append((x, y, z, i.__name__, taux_reussite))

    # Tri des résultats par taux de réussite (du plus élevé au plus faible)
    results_sorted = sorted(results, key=lambda x: x[4], reverse=True)
    
    # Affichage des résultats triés
    for res in results_sorted:
        print(f"{res[0]} - {res[1]} - {res[2]} - {res[3]} : {res[4]}")

possibilite()


C:\Users\d.truong\AppData\Local\Temp\ipykernel_12624\1930870088.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  self.data_brut = pd.read_json("train (1).jsonl", lines=True)[["texte_annonce", "cal_réponse_signalement"]]


ValueError: Unexpected character found when decoding 'true'

lemmatisation - vectorisation_ponderee - nmf - ForetAleatoire : 0.741
racinisation - vectorisation_ponderee - nmf - ForetAleatoire : 0.738
rien - vectorisation_ponderee - nmf - ForetAleatoire : 0.737
lemmatisation - vectorisation_ponderee - nmf - RegressionLogistique : 0.734
racinisation - vectorisation_simple - svd - ForetAleatoire : 0.734
racinisation - vectorisation_ponderee - nmf - RegressionLogistique : 0.734
rien - vectorisation_ponderee - nmf - RegressionLogistique : 0.734
lemmatisation - vectorisation_simple - svd - ForetAleatoire : 0.733
rien - vectorisation_simple - svd - ForetAleatoire : 0.732
lemmatisation - vectorisation_simple - svd - RegressionLogistique : 0.725
racinisation - vectorisation_simple - svd - RegressionLogistique : 0.723
rien - vectorisation_simple - svd - RegressionLogistique : 0.723
rien - vectorisation_ponderee - nmf - KPlusProchesVoisins : 0.667
lemmatisation - vectorisation_ponderee - nmf - KPlusProchesVoisins : 0.658
racinisation - vectorisation_ponderee - nmf - KPlusProchesVoisins : 0.657
racinisation - vectorisation_simple - svd - KPlusProchesVoisins : 0.651
lemmatisation - vectorisation_simple - svd - KPlusProchesVoisins : 0.648
rien - vectorisation_simple - svd - KPlusProchesVoisins : 0.644
lemmatisation - vectorisation_ponderee - nmf - ArbreDeDecision : 0.643
racinisation - vectorisation_ponderee - nmf - ArbreDeDecision : 0.640
rien - vectorisation_ponderee - nmf - ArbreDeDecision : 0.637
lemmatisation - vectorisation_simple - svd - ArbreDeDecision : 0.623
rien - vectorisation_simple - svd - ArbreDeDecision : 0.617
racinisation - vectorisation_simple - svd - ArbreDeDecision : 0.586
rien - vectorisation_ponderee - nmf - BayesienNaif : 0.494
lemmatisation - vectorisation_ponderee - nmf - BayesienNaif : 0.491
racinisation - vectorisation_ponderee - nmf - BayesienNaif : 0.483
lemmatisation - vectorisation_simple - svd - BayesienNaif : 0.329
rien - vectorisation_simple - svd - BayesienNaif : 0.328
racinisation - vectorisation_simple - svd - BayesienNaif : 0.326

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2', None]
}

data = Data()
data.lemmatisation("texte_annonce")
data.vectorisation_ponderee("texte_annonce")
data.reduction_nmf()
data = data.get_data()

foret = ForetAleatoire(data)
foret.recherche_hyperparametres(param_grid)

In [ ]:
data = Data()
data.lemmatisation("texte_annonce")
data.vectorisation_ponderee("texte_annonce")
data.reduction_nmf()
data = data.get_data()

foret = ForetAleatoire(data)
foret.entrainement(max_depth=20, max_features="sqrt", min_samples_leaf=2, min_samples_split=5, n_estimators=200)  
print("Taux de réussite :", foret.taux_reussite())
